In [295]:
import pandas as pd
import numpy as np
import matplotlib as plt

#Reading file and renaming columns
weather = pd.read_csv('C:/Users/shahi/Downloads/Weather_KNMI_Station 275_cropped_1990_2025.csv', skiprows=13)
weather.columns=weather.columns.str.strip()
weather.rename(columns={'FG':'Windspeed', 'TN':'Tmin', 'TX':'Tmax', 'RH':'Prcpn', 'UG':'RelHum'}, inplace=True)

#Converting object data types to float (other parameters are already in int64 type
weather['Windspeed']=pd.to_numeric(weather['Windspeed'], errors='coerce')
weather['Q']=pd.to_numeric(weather['Q'], errors='coerce')

#Separating date into year, month and date
weather["Year"]= weather.YYYYMMDD//10000
weather["Month"] = weather.YYYYMMDD%10000//100
weather["Date"] = weather.YYYYMMDD%10

#Converting units from KNMI data to APEX required format
#Tmin, Tmax in degree C, Prcpn in mm, Q in MJ/m2, Windspeed in m/s
weather.Tmin=weather.Tmin*0.1
weather.Tmax=weather.Tmax*0.1
weather["Prcpn"]=weather["Prcpn"].replace(-1,0)
weather.Prcpn = weather.Prcpn*0.1
weather.Windspeed=weather.Windspeed*0.1
weather.Q = weather.Q*0.01

weather.head()

,# STN,YYYYMMDD,Windspeed,Tmin,Tmax,Q,Prcpn,RelHum,Year,Month,Date
0,375,19900101,1.5,-0.3,0.5,NaN,0.0,97,1990,1,1
1,375,19900102,1.0,-1.5,1.8,NaN,0.0,100,1990,1,2
2,375,19900103,4.6,-0.3,2.1,NaN,0.0,97,1990,1,3
3,375,19900104,3.6,-2.0,1.2,NaN,0.1,95,1990,1,4
4,375,19900105,3.6,1.0,3.6,NaN,0.0,97,1990,1,5


In [296]:
# sad_mod = pd.read_csv("C:/Users/shahi/Downloads/TxtInOut_APEX1501_modified/TxtInOut/SITE14.SAD", skiprows=9, delimiter='\\s+')
# sad_org = pd.read_csv("C:/Users/shahi/Downloads/TxtInOut_APEX1501_exdata/TxtInOut/SITE14.SAD", skiprows = 9, delimiter='\\s+')

# #repeat same with exdata sad file and compare
# sad_org.head(20)

# sad_mod.plot(y='PRCP', ylabel="Precipitation (mm)")
# sad_org.plot(y='PRCP', ylabel= "Precipitation (mm)")

weather.dtypes

# STN          int64
YYYYMMDD       int64
Windspeed    float64
Tmin         float64
Tmax         float64
Q            float64
Prcpn        float64
RelHum         int64
Year           int64
Month          int64
Date           int64
dtype: object

In [297]:
#Avg monthly max-min air temperature, precipitation, windspeed, Q
monthly_weather = weather.groupby(weather['Month'])
monthly_avg = monthly_weather.mean()

monthly_avg

,# STN,YYYYMMDD,Windspeed,Tmin,Tmax,Q,Prcpn,RelHum,Year,Date
Month,,,,,,,,,,
1,375.0,2.007512e+07,4.550674,0.332885,6.008871,2.490313,2.103405,87.534050,2007.500000,4.387097
2,375.0,2.007526e+07,4.392330,0.425860,7.385939,4.774818,2.129302,84.379548,2007.504425,4.539823
3,375.0,2.007405e+07,4.004332,1.865162,11.043412,9.047123,1.647383,78.860108,2007.373646,4.382671
4,375.0,2.007042e+07,3.569238,3.970476,15.088857,14.438958,1.394952,73.208571,2007.000000,4.500000
5,375.0,2.007052e+07,3.372903,7.670138,19.144700,17.778054,1.885622,72.793548,2007.000000,4.387097
6,375.0,2.007062e+07,3.138762,10.661048,22.033524,19.357177,2.180952,73.625714,2007.000000,4.500000
7,375.0,2.007072e+07,3.129309,12.613088,23.945346,18.252500,2.253272,75.059908,2007.000000,4.387097
8,375.0,2.007082e+07,2.973180,12.261198,23.793088,15.639667,2.240829,77.117051,2007.000000,4.387097
9,375.0,2.007092e+07,3.113524,9.641905,20.032381,11.172417,2.051905,81.946667,2007.000000,4.500000


In [326]:
#Monthly average standard deviation of daily min and max temperature
monthly_stddev=monthly_weather[['Tmax', 'Tmin']].std()
monthly_stddev.rename(columns={'Tmax':'Tmax_SD', 'Tmin':'Tmin_SD'}, inplace=True)
monthly_stddev

,Tmax_SD,Tmin_SD
Month,,
1,4.159948,4.710538
2,4.374835,4.697003
3,4.148721,3.780439
4,4.555218,3.668099
5,4.607589,3.456703
6,4.374240,3.110681
7,4.349062,2.697545
8,4.089050,2.791846
9,3.743955,3.141059


In [328]:
#Average number of rain days per month
weather['Rainyday']= weather['Prcpn'] > 0
monthly_num_rainydays = weather.groupby(['Year', 'Month'])['Rainyday'].sum()
monthly_avg['Rainydays_avg']=monthly_num_rainydays.groupby(['Month']).mean()
monthly_avg

,# STN,YYYYMMDD,Windspeed,Tmin,Tmax,Q,Prcpn,RelHum,Year,Date,Rainydays_avg
Month,,,,,,,,,,,
1,375.0,2.007512e+07,4.550674,0.332885,6.008871,2.490313,2.103405,87.534050,2007.500000,4.387097,17.194444
2,375.0,2.007526e+07,4.392330,0.425860,7.385939,4.774818,2.129302,84.379548,2007.504425,4.539823,15.333333
3,375.0,2.007405e+07,4.004332,1.865162,11.043412,9.047123,1.647383,78.860108,2007.373646,4.382671,14.777778
4,375.0,2.007042e+07,3.569238,3.970476,15.088857,14.438958,1.394952,73.208571,2007.000000,4.500000,12.628571
5,375.0,2.007052e+07,3.372903,7.670138,19.144700,17.778054,1.885622,72.793548,2007.000000,4.387097,13.171429
6,375.0,2.007062e+07,3.138762,10.661048,22.033524,19.357177,2.180952,73.625714,2007.000000,4.500000,12.942857
7,375.0,2.007072e+07,3.129309,12.613088,23.945346,18.252500,2.253272,75.059908,2007.000000,4.387097,14.114286
8,375.0,2.007082e+07,2.973180,12.261198,23.793088,15.639667,2.240829,77.117051,2007.000000,4.387097,13.600000
9,375.0,2.007092e+07,3.113524,9.641905,20.032381,11.172417,2.051905,81.946667,2007.000000,4.500000,13.228571


In [332]:
#Transpose and save relevant columns in output
final_data=pd.concat([monthly_avg, monthly_stddev], axis=1).round(3)
final_data[["Q", "Tmax", "Tmin", "Prcpn", "Windspeed", "Rainydays_avg", "Tmax_SD", "Tmin_SD"]].T.to_csv('C:/Users/shahi/Downloads/Monthly_weather_stats_1990_2025.csv')
#monthly_stddev[["Tmax_SD", "Tmin_SD"]].T.to_csv('C:/Users/shahi/Downloads/Monthly_weather_stats_1990_2025.csv')